# TensorFlow

[How to setup tensorflow in Linux?](https://www.youtube.com/watch?v=6-eEpq7ChKg])

In [6]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


In [4]:
# Define the computational graph
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(5, name="X_Value")
    y = tf.constant(2, name="Y_value")
    sum = tf.add(x, y, name="xy_sum")

# Execute the graph
with tf.compat.v1.Session(graph=graph) as session:
    result = session.run(sum)
    print("Result:", result)

TensorFlow version: 2.15.0
Result: 7


## Load a dataset

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by 255.0. This also converts the sample data from integers to floating-point numbers:

In [8]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 1s 0us/step


### Build a ML model

In [9]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [10]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.11964792, -0.6828836 ,  0.0058978 ,  0.13633962,  1.0843248 ,
        -0.51743245, -0.45635572,  0.1578706 ,  0.34093377, -0.1590127 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class

In [11]:
tf.nn.softmax(predictions).numpy()

array([[0.0794889 , 0.04525811, 0.09012191, 0.10267874, 0.26496303,
        0.05340119, 0.05676442, 0.10491348, 0.12598959, 0.07642063]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. 

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [12]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [13]:
loss_fn(y_train[:1], predictions).numpy()

2.929922

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [14]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model


Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [15]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 984us/step - loss: 0.2929 - accuracy: 0.9143
Epoch 2/5
1875/1875 [==============================] - 2s 953us/step - loss: 0.1406 - accuracy: 0.9584
Epoch 3/5
1875/1875 [==============================] - 2s 944us/step - loss: 0.1061 - accuracy: 0.9674
Epoch 4/5
1875/1875 [==============================] - 2s 951us/step - loss: 0.0883 - accuracy: 0.9713
Epoch 5/5
1875/1875 [==============================] - 2s 942us/step - loss: 0.0755 - accuracy: 0.9762


The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [16]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0745 - accuracy: 0.9760 - 266ms/epoch - 849us/step


[0.07445836067199707, 0.9760000109672546]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [17]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [18]:
probability_model(x_test[:5])


<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.58860995e-08, 3.00899639e-10, 2.96819758e-06, 6.19027705e-05,
        1.72523877e-12, 3.72573993e-07, 1.09804289e-13, 9.99907970e-01,
        1.13520161e-06, 2.55924988e-05],
       [1.00929333e-06, 1.17589093e-06, 9.99980807e-01, 1.63351069e-05,
        5.04819548e-13, 2.94332505e-08, 2.46685516e-08, 2.25993130e-11,
        6.87515978e-07, 4.42640195e-11],
       [5.22985530e-08, 9.99171853e-01, 9.60963280e-06, 1.06722780e-06,
        5.94946869e-05, 1.98578055e-05, 3.22952997e-06, 7.00317149e-04,
        3.22923006e-05, 2.15989303e-06],
       [9.99967933e-01, 3.03812808e-10, 4.50338985e-06, 2.04289208e-09,
        1.05218510e-06, 2.26054567e-06, 8.99289716e-06, 5.32827335e-06,
        1.18374164e-08, 9.96654308e-06],
       [3.42325948e-05, 8.82923601e-09, 6.71275411e-05, 1.52464580e-07,
        9.84135211e-01, 8.75094997e-07, 1.43368584e-06, 1.38704578e-04,
        2.17345405e-06, 1.56201245e-02]], dtype=float32)>